In [1]:
mkdir single_subject

In [2]:
segstats2nidm -add_de -s `pwd`/subjects/Olin_0050107 -o `pwd`/single_subject

Creating NIDM file...
Writing NIDM file...
Creating NIDM file...
Writing NIDM file...
Creating NIDM file...
Writing NIDM file...


In [3]:
ls single_subject

aseg_nidm.ttl		lh.aparc_nidm.ttl	rh.aparc_nidm.ttl


In [4]:
pynidm query -nl `pwd`/single_subject/aseg_nidm.ttl -bv -o `pwd`/single_query_results.txt

In [5]:
cat `pwd`/single_query_results.txt

,ID,tool,softwareLabel,federatedLabel,laterality,volume
0,Olin_0050107,https://surfer.nmr.mgh.harvard.edu/,Left-Putamen Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001874,Left,6727.1
1,Olin_0050107,https://surfer.nmr.mgh.harvard.edu/,4th-Ventricle Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0002422,,2383.2
2,Olin_0050107,https://surfer.nmr.mgh.harvard.edu/,non-WM-hypointensities Volume_mm3 (mm^3),,,0.0
3,Olin_0050107,https://surfer.nmr.mgh.harvard.edu/,Right-Cerebellum-Cortex Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0002129,Right,68967.0
4,Olin_0050107,https://surfer.nmr.mgh.harvard.edu/,CC_Central Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0002336,,942.4
5,Olin_0050107,https://surfer.nmr.mgh.harvard.edu/,3rd-Ventricle Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0002286,,740.6
6,Olin_0050107,https://surfer.nmr.mgh.harvard.edu/,Left-vessel Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001981,Left,70.7
7,Olin_005

60,Olin_0050107,https://surfer.nmr.mgh.harvard.edu/,Right-Accumbens-area Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001882,Right,801.2
61,Olin_0050107,https://surfer.nmr.mgh.harvard.edu/,Right hemisphere cerebral white matter volume (mm^3),http://purl.obolibrary.org/obo/UBERON_0002437,Right,288733.501093


In [6]:
mkdir all_subjects

In [7]:
for subject_dir in subjects/*
do
    subject=`basename $subject_dir`
    echo $subject
    mkdir all_subjects/$subject
    segstats2nidm -add_de -s `pwd`/$subject_dir -o `pwd`/all_subjects/$subject
    echo
done
echo done

CMU_0050646
Creating NIDM file...
Writing NIDM file...
Creating NIDM file...
Writing NIDM file...
Creating NIDM file...
Writing NIDM file...

Caltech_0051456
Creating NIDM file...
Writing NIDM file...
Creating NIDM file...
Writing NIDM file...
Creating NIDM file...
Writing NIDM file...

KKI_0050778
Creating NIDM file...
Writing NIDM file...
Creating NIDM file...
Writing NIDM file...
Creating NIDM file...
Writing NIDM file...

Leuven_0050691
Creating NIDM file...
Writing NIDM file...
Creating NIDM file...
Writing NIDM file...
Creating NIDM file...
Writing NIDM file...

MaxMun_0051321
Creating NIDM file...
Writing NIDM file...
Creating NIDM file...
Writing NIDM file...
Creating NIDM file...
Writing NIDM file...

NYU_0050965
Creating NIDM file...
Writing NIDM file...
Creating NIDM file...
Writing NIDM file...
Creating NIDM file...
Writing NIDM file...

OHSU_0050142
Creating NIDM file...
Writing NIDM file...
Creating NIDM file...
Writing NIDM file...
Creating NIDM file...
Writing NIDM file

In [8]:
aseg_list=`echo all_subjects/*/aseg_nidm.ttl | tr ' ' ,`

In [9]:
pynidm concat -nl $aseg_list -o all_subjects/all_aseg.ttl

In [10]:
pynidm query -nl all_subjects/all_aseg.ttl -bv -o all_query_results.txt

In [11]:
cat all_query_results.txt

,ID,tool,softwareLabel,federatedLabel,laterality,volume
0,NYU_0050965,https://surfer.nmr.mgh.harvard.edu/,Right-VentralDC Volume_mm3 (mm^3),,Right,3707.3
1,Olin_0050107,https://surfer.nmr.mgh.harvard.edu/,Right-VentralDC Volume_mm3 (mm^3),,Right,4620.4
2,Caltech_0051456,https://surfer.nmr.mgh.harvard.edu/,Right-VentralDC Volume_mm3 (mm^3),,Right,4184.7
3,Trinity_0050240,https://surfer.nmr.mgh.harvard.edu/,Right-VentralDC Volume_mm3 (mm^3),,Right,4378.5
4,KKI_0050778,https://surfer.nmr.mgh.harvard.edu/,Right-VentralDC Volume_mm3 (mm^3),,Right,3639.3
5,SBL_0051556,https://surfer.nmr.mgh.harvard.edu/,Right-VentralDC Volume_mm3 (mm^3),,Right,4477.9
6,UM_0050404,https://surfer.nmr.mgh.harvard.edu/,Right-VentralDC Volume_mm3 (mm^3),,Right,2137.1
7,Pitt_0050003,https://surfer.nmr.mgh.harvard.edu/,Right-VentralDC Volume_mm3 (mm^3),,Right,5173.1
8,UCLA_0051210,https://surfer.nmr.mgh.harvard.edu/,Right-VentralDC Volume_mm3 (mm^3),,Right,3923.3
9,CMU_0050646,https://surfer.nmr.mgh.harvard.edu/,Ri

64,OHSU_0050142,https://surfer.nmr.mgh.harvard.edu/,Brain Segmentation Volume Without Ventricles from Surf (mm^3),http://purl.obolibrary.org/obo/UBERON_0000955,,1323230.682649
65,Leuven_0050691,https://surfer.nmr.mgh.harvard.edu/,Brain Segmentation Volume Without Ventricles from Surf (mm^3),http://purl.obolibrary.org/obo/UBERON_0000955,,1322124.021799
66,Stanford_0051168,https://surfer.nmr.mgh.harvard.edu/,Brain Segmentation Volume Without Ventricles from Surf (mm^3),http://purl.obolibrary.org/obo/UBERON_0000955,,1276245.402709
67,USM_0050440,https://surfer.nmr.mgh.harvard.edu/,Brain Segmentation Volume Without Ventricles from Surf (mm^3),http://purl.obolibrary.org/obo/UBERON_0000955,,1244714.612594
68,NYU_0050965,https://surfer.nmr.mgh.harvard.edu/,CC_Posterior Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0015708,,987.0
69,Olin_0050107,https://surfer.nmr.mgh.harvard.edu/,CC_Posterior Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0015708,,1252.0
70,Caltech_0051456,

120,Olin_0050107,https://surfer.nmr.mgh.harvard.edu/,Optic-Chiasm Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0000959,,85.9
121,Caltech_0051456,https://surfer.nmr.mgh.harvard.edu/,Optic-Chiasm Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0000959,,133.7
122,Trinity_0050240,https://surfer.nmr.mgh.harvard.edu/,Optic-Chiasm Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0000959,,148.6
123,KKI_0050778,https://surfer.nmr.mgh.harvard.edu/,Optic-Chiasm Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0000959,,190.7
124,SBL_0051556,https://surfer.nmr.mgh.harvard.edu/,Optic-Chiasm Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0000959,,141.8
125,UM_0050404,https://surfer.nmr.mgh.harvard.edu/,Optic-Chiasm Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0000959,,75.0
126,Pitt_0050003,https://surfer.nmr.mgh.harvard.edu/,Optic-Chiasm Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0000959,,130.0
127,UCLA_0051210,https://surfer.nmr.mgh.h

176,UM_0050404,https://surfer.nmr.mgh.harvard.edu/,Right hemisphere cortical gray matter volume (mm^3),http://purl.obolibrary.org/obo/UBERON_0000956,Right,196074.531158
177,Pitt_0050003,https://surfer.nmr.mgh.harvard.edu/,Right hemisphere cortical gray matter volume (mm^3),http://purl.obolibrary.org/obo/UBERON_0000956,Right,255435.863143
178,UCLA_0051210,https://surfer.nmr.mgh.harvard.edu/,Right hemisphere cortical gray matter volume (mm^3),http://purl.obolibrary.org/obo/UBERON_0000956,Right,278348.051221
179,CMU_0050646,https://surfer.nmr.mgh.harvard.edu/,Right hemisphere cortical gray matter volume (mm^3),http://purl.obolibrary.org/obo/UBERON_0000956,Right,292112.860343
180,Yale_0050559,https://surfer.nmr.mgh.harvard.edu/,Right hemisphere cortical gray matter volume (mm^3),http://purl.obolibrary.org/obo/UBERON_0000956,Right,281167.557428
181,SDSU_0050185,https://surfer.nmr.mgh.harvard.edu/,Right hemisphere cortical gray matter volume (mm^3),http://purl.obolibrary.org/obo/UBERON_00009

235,Leuven_0050691,https://surfer.nmr.mgh.harvard.edu/,Mask Volume (mm^3),,,1904522.0
236,Stanford_0051168,https://surfer.nmr.mgh.harvard.edu/,Mask Volume (mm^3),,,1748768.0
237,USM_0050440,https://surfer.nmr.mgh.harvard.edu/,Mask Volume (mm^3),,,1746354.0
238,NYU_0050965,https://surfer.nmr.mgh.harvard.edu/,CC_Central Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0002336,,628.6
239,Olin_0050107,https://surfer.nmr.mgh.harvard.edu/,CC_Central Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0002336,,942.4
240,Caltech_0051456,https://surfer.nmr.mgh.harvard.edu/,CC_Central Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0002336,,666.9
241,Trinity_0050240,https://surfer.nmr.mgh.harvard.edu/,CC_Central Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0002336,,947.3
242,KKI_0050778,https://surfer.nmr.mgh.harvard.edu/,CC_Central Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0002336,,551.5
243,SBL_0051556,https://surfer.nmr.mgh.harvard.edu/,CC_Central V

...

770,SBL_0051556,https://surfer.nmr.mgh.harvard.edu/,Right-Thalamus-Proper Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001897,Right,7598.5
771,UM_0050404,https://surfer.nmr.mgh.harvard.edu/,Right-Thalamus-Proper Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001897,Right,4926.1
772,Pitt_0050003,https://surfer.nmr.mgh.harvard.edu/,Right-Thalamus-Proper Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001897,Right,9432.6
773,UCLA_0051210,https://surfer.nmr.mgh.harvard.edu/,Right-Thalamus-Proper Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001897,Right,7326.9
774,CMU_0050646,https://surfer.nmr.mgh.harvard.edu/,Right-Thalamus-Proper Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001897,Right,8792.9
775,Yale_0050559,https://surfer.nmr.mgh.harvard.edu/,Right-Thalamus-Proper Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001897,Right,7598.0
776,SDSU_0050185,https://surfer.nmr.mgh.harvard.edu/,Right-Thalamus-Proper Volume_mm3 (mm^3)

827,SDSU_0050185,https://surfer.nmr.mgh.harvard.edu/,3rd-Ventricle Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0002286,,703.8
828,MaxMun_0051321,https://surfer.nmr.mgh.harvard.edu/,3rd-Ventricle Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0002286,,894.1
829,OHSU_0050142,https://surfer.nmr.mgh.harvard.edu/,3rd-Ventricle Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0002286,,974.4
830,Leuven_0050691,https://surfer.nmr.mgh.harvard.edu/,3rd-Ventricle Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0002286,,955.5
831,Stanford_0051168,https://surfer.nmr.mgh.harvard.edu/,3rd-Ventricle Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0002286,,903.9
832,USM_0050440,https://surfer.nmr.mgh.harvard.edu/,3rd-Ventricle Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0002286,,1363.7
833,NYU_0050965,https://surfer.nmr.mgh.harvard.edu/,CC_Anterior Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0015599,,1078.7
834,Olin_0050107,https://sur

891,Pitt_0050003,https://surfer.nmr.mgh.harvard.edu/,Right-Caudate Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001873,Right,3536.1
892,UCLA_0051210,https://surfer.nmr.mgh.harvard.edu/,Right-Caudate Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001873,Right,3536.2
893,CMU_0050646,https://surfer.nmr.mgh.harvard.edu/,Right-Caudate Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001873,Right,3475.9
894,Yale_0050559,https://surfer.nmr.mgh.harvard.edu/,Right-Caudate Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001873,Right,3860.9
895,SDSU_0050185,https://surfer.nmr.mgh.harvard.edu/,Right-Caudate Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001873,Right,4754.1
896,MaxMun_0051321,https://surfer.nmr.mgh.harvard.edu/,Right-Caudate Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001873,Right,3183.9
897,OHSU_0050142,https://surfer.nmr.mgh.harvard.edu/,Right-Caudate Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001873,Righ

947,MaxMun_0051321,https://surfer.nmr.mgh.harvard.edu/,Brain-Stem Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0002298,,20383.9
948,OHSU_0050142,https://surfer.nmr.mgh.harvard.edu/,Brain-Stem Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0002298,,20961.3
949,Leuven_0050691,https://surfer.nmr.mgh.harvard.edu/,Brain-Stem Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0002298,,23473.4
950,Stanford_0051168,https://surfer.nmr.mgh.harvard.edu/,Brain-Stem Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0002298,,17204.8
951,USM_0050440,https://surfer.nmr.mgh.harvard.edu/,Brain-Stem Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0002298,,20625.4
952,NYU_0050965,https://surfer.nmr.mgh.harvard.edu/,Left-Hippocampus Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001954,Left,3970.7
953,Olin_0050107,https://surfer.nmr.mgh.harvard.edu/,Left-Hippocampus Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001954,Left,4318.4
954,Caltech_00514

1013,Yale_0050559,https://surfer.nmr.mgh.harvard.edu/,Right-Inf-Lat-Vent Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0006091,Right,218.9
1014,SDSU_0050185,https://surfer.nmr.mgh.harvard.edu/,Right-Inf-Lat-Vent Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0006091,Right,184.1
1015,MaxMun_0051321,https://surfer.nmr.mgh.harvard.edu/,Right-Inf-Lat-Vent Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0006091,Right,340.2
1016,OHSU_0050142,https://surfer.nmr.mgh.harvard.edu/,Right-Inf-Lat-Vent Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0006091,Right,246.3
1017,Leuven_0050691,https://surfer.nmr.mgh.harvard.edu/,Right-Inf-Lat-Vent Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0006091,Right,341.4
1018,Stanford_0051168,https://surfer.nmr.mgh.harvard.edu/,Right-Inf-Lat-Vent Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0006091,Right,460.0
1019,USM_0050440,https://surfer.nmr.mgh.harvard.edu/,Right-Inf-Lat-Vent Volume_mm3 (mm^3),http://p

In [12]:
pynidm query -nl all_subjects/all_aseg.ttl -q hippocampus_query.sparql -o all_hippocampus_results.txt

In [13]:
cat all_hippocampus_results.txt

,ID,tool,softwareLabel,federatedLabel,laterality,volume
0,NYU_0050965,https://surfer.nmr.mgh.harvard.edu/,Right-Hippocampus Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001954,Right,3912.0
1,Olin_0050107,https://surfer.nmr.mgh.harvard.edu/,Right-Hippocampus Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001954,Right,4539.9
2,Caltech_0051456,https://surfer.nmr.mgh.harvard.edu/,Right-Hippocampus Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001954,Right,4487.2
3,Trinity_0050240,https://surfer.nmr.mgh.harvard.edu/,Right-Hippocampus Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001954,Right,4568.3
4,KKI_0050778,https://surfer.nmr.mgh.harvard.edu/,Right-Hippocampus Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001954,Right,3797.7
5,SBL_0051556,https://surfer.nmr.mgh.harvard.edu/,Right-Hippocampus Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001954,Right,3793.6
6,UM_0050404,https://surfer.nmr.mgh.harvard.edu/,Right-Hippocampu

`csv2nidm -csv demographics.csv -out demographics.ttl` run earlier

In [14]:
pynidm merge -nl demographics.ttl,all_subjects/all_aseg.ttl -s -o demog_aseg.ttl

In [15]:
pynidm query -nl demog_aseg.ttl -u /projects

UUID
------------------------------------
fa1810dc-866a-11ea-ae3c-f01898563600


In [16]:
pynidm query -nl demog_aseg.ttl -u /projects/fa1810dc-866a-11ea-ae3c-f01898563600

-----------------------------------------------  ------------------------------------
AcquisitionModality                              []
ImageContrastType                                []
ImageUsageType                                   []
Task                                             []
http://www.w3.org/1999/02/22-rdf-syntax-ns#type  "http://www.w3.org/ns/prov#Activity"
nfo:filename                                     "demographics.csv"
-----------------------------------------------  ------------------------------------

Subject Information
---------------------
uuid                                  subject id
------------------------------------  ----------------
fa1cdf40-866a-11ea-ae3c-f01898563600  CMU_0050646
fa1f25c0-866a-11ea-ae3c-f01898563600  Caltech_0051456
fa231266-866a-11ea-ae3c-f01898563600  KKI_0050778
fa213810-866a-11ea-ae3c-f01898563600  Leuven_0050691
fa24f4aa-866a-11ea-ae3c-f01898563600  MaxMun_0051321
fa26d3ce-866a-11ea-ae3c-f01898563600  NYU_0050965
fa28d444-

fs_002399           Left BA4a_exvivo SurfArea (mm^2)
fs_002401           Left BA4a_exvivo ThickAvg (mm)
fs_002402           Left BA4a_exvivo ThickStd (mm)
fs_002415           Left BA4p_exvivo CurvInd
fs_002414           Left BA4p_exvivo FoldInd
fs_002413           Left BA4p_exvivo GausCurv (mm^-2)
fs_002409           Left BA4p_exvivo GrayVol (mm^3)
fs_002412           Left BA4p_exvivo MeanCurv (mm^-1)
fs_002407           Left BA4p_exvivo NumVert
fs_002408           Left BA4p_exvivo SurfArea (mm^2)
fs_002410           Left BA4p_exvivo ThickAvg (mm)
fs_002411           Left BA4p_exvivo ThickStd (mm)
fs_002424           Left BA6_exvivo CurvInd
fs_002423           Left BA6_exvivo FoldInd
fs_002422           Left BA6_exvivo GausCurv (mm^-2)
fs_002418           Left BA6_exvivo GrayVol (mm^3)
fs_002421           Left BA6_exvivo MeanCurv (mm^-1)
fs_002416           Left BA6_exvivo NumVert
fs_002417           Left BA6_exvivo SurfArea (mm^2)
fs_002419           Left BA6_exvivo ThickAvg (mm)
fs_0

fs_001819           Left G_front_inf-Triangul GausCurv (mm^-2)
fs_001815           Left G_front_inf-Triangul GrayVol (mm^3)
fs_001818           Left G_front_inf-Triangul MeanCurv (mm^-1)
fs_001813           Left G_front_inf-Triangul NumVert
fs_001814           Left G_front_inf-Triangul SurfArea (mm^2)
fs_001816           Left G_front_inf-Triangul ThickAvg (mm)
fs_001817           Left G_front_inf-Triangul ThickStd (mm)
fs_001830           Left G_front_middle CurvInd
fs_001829           Left G_front_middle FoldInd
fs_001828           Left G_front_middle GausCurv (mm^-2)
fs_001824           Left G_front_middle GrayVol (mm^3)
fs_001827           Left G_front_middle MeanCurv (mm^-1)
fs_001822           Left G_front_middle NumVert
fs_001823           Left G_front_middle SurfArea (mm^2)
fs_001825           Left G_front_middle ThickAvg (mm)
fs_001826           Left G_front_middle ThickStd (mm)
fs_001839           Left G_front_sup CurvInd
fs_001838           Left G_front_sup FoldInd
fs_001837 

fs_001983           Left G_subcallosal CurvInd
fs_001982           Left G_subcallosal FoldInd
fs_001981           Left G_subcallosal GausCurv (mm^-2)
fs_001977           Left G_subcallosal GrayVol (mm^3)
fs_001980           Left G_subcallosal MeanCurv (mm^-1)
fs_001975           Left G_subcallosal NumVert
fs_001976           Left G_subcallosal SurfArea (mm^2)
fs_001978           Left G_subcallosal ThickAvg (mm)
fs_001979           Left G_subcallosal ThickStd (mm)
fs_001992           Left G_temp_sup-G_T_transv CurvInd
fs_001991           Left G_temp_sup-G_T_transv FoldInd
fs_001990           Left G_temp_sup-G_T_transv GausCurv (mm^-2)
fs_001986           Left G_temp_sup-G_T_transv GrayVol (mm^3)
fs_001989           Left G_temp_sup-G_T_transv MeanCurv (mm^-1)
fs_001984           Left G_temp_sup-G_T_transv NumVert
fs_001985           Left G_temp_sup-G_T_transv SurfArea (mm^2)
fs_001987           Left G_temp_sup-G_T_transv ThickAvg (mm)
fs_001988           Left G_temp_sup-G_T_transv ThickS

fs_002111           Left S_circular_insula_ant SurfArea (mm^2)
fs_002113           Left S_circular_insula_ant ThickAvg (mm)
fs_002114           Left S_circular_insula_ant ThickStd (mm)
fs_002127           Left S_circular_insula_inf CurvInd
fs_002126           Left S_circular_insula_inf FoldInd
fs_002125           Left S_circular_insula_inf GausCurv (mm^-2)
fs_002121           Left S_circular_insula_inf GrayVol (mm^3)
fs_002124           Left S_circular_insula_inf MeanCurv (mm^-1)
fs_002119           Left S_circular_insula_inf NumVert
fs_002120           Left S_circular_insula_inf SurfArea (mm^2)
fs_002122           Left S_circular_insula_inf ThickAvg (mm)
fs_002123           Left S_circular_insula_inf ThickStd (mm)
fs_002136           Left S_circular_insula_sup CurvInd
fs_002135           Left S_circular_insula_sup FoldInd
fs_002134           Left S_circular_insula_sup GausCurv (mm^-2)
fs_002130           Left S_circular_insula_sup GrayVol (mm^3)
fs_002133           Left S_circular_ins

...

fs_000367           Right rostralmiddlefrontal ThickStd (mm)
fs_002697           Right superiorfrontal Area_mm2 (mm^2)
fs_000380           Right superiorfrontal CurvInd
fs_000379           Right superiorfrontal FoldInd
fs_000378           Right superiorfrontal GausCurv (mm^-2)
fs_000374           Right superiorfrontal GrayVol (mm^3)
fs_000377           Right superiorfrontal MeanCurv (mm^-1)
fs_002696           Right superiorfrontal NVertices
fs_000372           Right superiorfrontal NumVert
fs_002703           Right superiorfrontal SNR (none)
fs_000373           Right superiorfrontal SurfArea (mm^2)
fs_000375           Right superiorfrontal ThickAvg (mm)
fs_000376           Right superiorfrontal ThickStd (mm)
fs_002705           Right superiorparietal Area_mm2 (mm^2)
fs_000389           Right superiorparietal CurvInd
fs_000388           Right superiorparietal FoldInd
fs_000387           Right superiorparietal GausCurv (mm^-2)
fs_000383           Right superiorparietal GrayVol (mm^3)
fs

fs_003253           Right-UnsegmentedWhiteMatter normMax (MR)
fs_003250           Right-UnsegmentedWhiteMatter normMean (MR)
fs_003252           Right-UnsegmentedWhiteMatter normMin (MR)
fs_003254           Right-UnsegmentedWhiteMatter normRange (MR)
fs_003251           Right-UnsegmentedWhiteMatter normStdDev (MR)
fs_003468           Right-VentralDC NVoxels
fs_003469           Right-VentralDC Volume_mm3 (mm^3)
fs_003473           Right-VentralDC normMax (MR)
fs_003470           Right-VentralDC normMean (MR)
fs_003472           Right-VentralDC normMin (MR)
fs_003474           Right-VentralDC normRange (MR)
fs_003471           Right-VentralDC normStdDev (MR)
fs_003510           Right-WM-hypointensities NVoxels
fs_003511           Right-WM-hypointensities Volume_mm3 (mm^3)
fs_003515           Right-WM-hypointensities normMax (MR)
fs_003512           Right-WM-hypointensities normMean (MR)
fs_003514           Right-WM-hypointensities normMin (MR)
fs_003516           Right-WM-hypointensities

fs_002835           wm-lh-lateralorbitofrontal NVoxels
fs_002836           wm-lh-lateralorbitofrontal Volume_mm3 (mm^3)
fs_002840           wm-lh-lateralorbitofrontal normMax (MR)
fs_002837           wm-lh-lateralorbitofrontal normMean (MR)
fs_002839           wm-lh-lateralorbitofrontal normMin (MR)
fs_002841           wm-lh-lateralorbitofrontal normRange (MR)
fs_002838           wm-lh-lateralorbitofrontal normStdDev (MR)
fs_002842           wm-lh-lingual NVoxels
fs_002843           wm-lh-lingual Volume_mm3 (mm^3)
fs_002847           wm-lh-lingual normMax (MR)
fs_002844           wm-lh-lingual normMean (MR)
fs_002846           wm-lh-lingual normMin (MR)
fs_002848           wm-lh-lingual normRange (MR)
fs_002845           wm-lh-lingual normStdDev (MR)
fs_002849           wm-lh-medialorbitofrontal NVoxels
fs_002850           wm-lh-medialorbitofrontal Volume_mm3 (mm^3)
fs_002854           wm-lh-medialorbitofrontal normMax (MR)
fs_002851           wm-lh-medialorbitofrontal normMean (MR)
fs

fs_002990           wm-lh-transversetemporal Volume_mm3 (mm^3)
fs_002994           wm-lh-transversetemporal normMax (MR)
fs_002991           wm-lh-transversetemporal normMean (MR)
fs_002993           wm-lh-transversetemporal normMin (MR)
fs_002995           wm-lh-transversetemporal normRange (MR)
fs_002992           wm-lh-transversetemporal normStdDev (MR)
fs_003003           wm-rh-bankssts NVoxels
fs_003004           wm-rh-bankssts Volume_mm3 (mm^3)
fs_003008           wm-rh-bankssts normMax (MR)
fs_003005           wm-rh-bankssts normMean (MR)
fs_003007           wm-rh-bankssts normMin (MR)
fs_003009           wm-rh-bankssts normRange (MR)
fs_003006           wm-rh-bankssts normStdDev (MR)
fs_003010           wm-rh-caudalanteriorcingulate NVoxels
fs_003011           wm-rh-caudalanteriorcingulate Volume_mm3 (mm^3)
fs_003015           wm-rh-caudalanteriorcingulate normMax (MR)
fs_003012           wm-rh-caudalanteriorcingulate normMean (MR)
fs_003014           wm-rh-caudalanteriorcingul

fs_003135           wm-rh-parstriangularis normRange (MR)
fs_003132           wm-rh-parstriangularis normStdDev (MR)
fs_003136           wm-rh-pericalcarine NVoxels
fs_003137           wm-rh-pericalcarine Volume_mm3 (mm^3)
fs_003141           wm-rh-pericalcarine normMax (MR)
fs_003138           wm-rh-pericalcarine normMean (MR)
fs_003140           wm-rh-pericalcarine normMin (MR)
fs_003142           wm-rh-pericalcarine normRange (MR)
fs_003139           wm-rh-pericalcarine normStdDev (MR)
fs_003143           wm-rh-postcentral NVoxels
fs_003144           wm-rh-postcentral Volume_mm3 (mm^3)
fs_003148           wm-rh-postcentral normMax (MR)
fs_003145           wm-rh-postcentral normMean (MR)
fs_003147           wm-rh-postcentral normMin (MR)
fs_003149           wm-rh-postcentral normRange (MR)
fs_003146           wm-rh-postcentral normStdDev (MR)
fs_003150           wm-rh-posteriorcingulate NVoxels
fs_003151           wm-rh-posteriorcingulate Volume_mm3 (mm^3)
fs_003155           wm-rh-p